# Izgradnja regresijskog modela: priprema i vizualizacija podataka

## **Linearna regresija za bundeve - Lekcija 2**
#### Uvod

Sada kada imate alate potrebne za izgradnju modela strojnog učenja koristeći Tidymodels i Tidyverse, spremni ste početi postavljati pitanja o svojim podacima. Dok radite s podacima i primjenjujete rješenja strojnog učenja, vrlo je važno znati kako postaviti pravo pitanje kako biste pravilno iskoristili potencijale svog skupa podataka.

U ovoj lekciji naučit ćete:

-   Kako pripremiti podatke za izgradnju modela.

-   Kako koristiti `ggplot2` za vizualizaciju podataka.

Pitanje na koje trebate odgovoriti odredit će koju vrstu algoritama strojnog učenja ćete koristiti. Kvaliteta odgovora koji dobijete uvelike će ovisiti o prirodi vaših podataka.

Pogledajmo ovo kroz praktičnu vježbu.


<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Umjetničko djelo @allison_horst</figcaption>


<!--![Umjetničko djelo \@allison_horst](../../../../../../2-Regression/2-Data/images/unruly_data.jpg)<br>Umjetničko djelo \@allison_horst-->


## 1. Uvoz podataka o bundevama i pozivanje Tidyverse-a

Trebat ćemo sljedeće pakete za analizu i obradu podataka u ovoj lekciji:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) je [kolekcija R paketa](https://www.tidyverse.org/packages) osmišljena kako bi znanost o podacima bila brža, jednostavnija i zabavnija!

Možete ih instalirati pomoću:

`install.packages(c("tidyverse"))`

Skripta ispod provjerava imate li instalirane potrebne pakete za dovršetak ovog modula i instalira ih za vas ako neki nedostaju.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Sada pokrenimo neke pakete i učitajmo [podatke](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv) koji su osigurani za ovu lekciju!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Brzi `glimpse()` odmah pokazuje da postoje praznine i mješavina stringova (`chr`) i numeričkih podataka (`dbl`). `Date` je tipa karakter, a tu je i čudan stupac nazvan `Package` gdje su podaci mješavina između `sacks`, `bins` i drugih vrijednosti. Podaci su, zapravo, pomalo neuredni 😤.

Zapravo, nije baš uobičajeno dobiti dataset koji je potpuno spreman za korištenje u stvaranju ML modela odmah iz kutije. Ali ne brinite, u ovoj lekciji naučit ćete kako pripremiti sirovi dataset koristeći standardne R biblioteke 🧑‍🔧. Također ćete naučiti razne tehnike za vizualizaciju podataka.📈📊
<br>

> Podsjetnik: Operator cijevi (`%>%`) izvodi operacije u logičnom slijedu prosljeđujući objekt dalje u funkciju ili izraz poziva. Možete zamisliti operator cijevi kao da u vašem kodu kaže "i onda".


## 2. Provjera nedostajućih podataka

Jedan od najčešćih problema s kojima se znanstvenici podataka susreću je nepotpuni ili nedostajući podaci. R predstavlja nedostajuće ili nepoznate vrijednosti posebnim sentinel vrijednostima: `NA` (Not Available).

Kako možemo znati da podatkovni okvir sadrži nedostajuće vrijednosti?
<br>
-   Jedan jednostavan način bio bi korištenje osnovne R funkcije `anyNA`, koja vraća logičke objekte `TRUE` ili `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Odlično, čini se da nedostaju neki podaci! To je dobro mjesto za početak.

-   Drugi način bio bi korištenje funkcije `is.na()` koja pokazuje koji su pojedinačni elementi stupca nedostajući s logičkom vrijednošću `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

U redu, posao je obavljen, ali s velikim podatkovnim okvirom poput ovog, bilo bi neučinkovito i praktički nemoguće pregledati sve redove i stupce pojedinačno😴.

-   Intuitivniji način bio bi izračunati zbroj nedostajućih vrijednosti za svaki stupac:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Mnogo bolje! Nedostaju neki podaci, ali možda to neće biti važno za zadatak koji je pred nama. Vidjet ćemo što će donijeti daljnja analiza.

> Uz sjajne skupove paketa i funkcija, R ima vrlo dobru dokumentaciju. Na primjer, koristite `help(colSums)` ili `?colSums` kako biste saznali više o funkciji.


## 3. Dplyr: Gramatika za manipulaciju podacima

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Ilustracija: @allison_horst</figcaption>


<!--![Ilustracija: \@allison_horst](../../../../../../2-Regression/2-Data/images/dplyr_wrangling.png)<br/>Ilustracija: \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), paket u Tidyverseu, predstavlja gramatiku manipulacije podacima koja pruža dosljedan skup glagola za rješavanje najčešćih izazova u manipulaciji podacima. U ovom dijelu istražit ćemo neke od dplyr-ovih glagola!


#### dplyr::select()

`select()` je funkcija u paketu `dplyr` koja vam pomaže odabrati stupce koje želite zadržati ili isključiti.

Kako biste olakšali rad s vašim podatkovnim okvirom, uklonite nekoliko njegovih stupaca koristeći `select()`, zadržavajući samo one stupce koji su vam potrebni.

Na primjer, u ovoj vježbi naša analiza će uključivati stupce `Package`, `Low Price`, `High Price` i `Date`. Odaberimo te stupce.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` je funkcija u paketu `dplyr` koja vam omogućuje stvaranje ili izmjenu stupaca, dok postojeći stupci ostaju nepromijenjeni.

Opća struktura funkcije mutate je:

`data %>%   mutate(new_column_name = what_it_contains)`

Hajdemo isprobati `mutate` koristeći stupac `Date` i izvršiti sljedeće operacije:

1.  Pretvoriti datume (trenutno tipa karakter) u format mjeseca (ovo su datumi iz SAD-a, pa je format `MM/DD/YYYY`).

2.  Izvući mjesec iz datuma u novi stupac.

U R-u, paket [lubridate](https://lubridate.tidyverse.org/) olakšava rad s podacima o datumu i vremenu. Dakle, koristit ćemo `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` i vidjeti kako postići gore navedene ciljeve. Možemo ukloniti stupac Date jer nam više neće biti potreban u daljnjim operacijama.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Jupi! 🤩

Sljedeće, kreirajmo novi stupac `Price`, koji predstavlja prosječnu cijenu bundeve. Sada, izračunajmo prosjek stupaca `Low Price` i `High Price` kako bismo popunili novi stupac Price.  
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Yeees!💪

"Ali čekaj!", reći ćeš nakon što brzo pregledaš cijeli skup podataka s `View(pumpkins)`, "Ovdje nešto ne štima!"🤔

Ako pogledaš stupac `Package`, bundeve se prodaju u mnogim različitim konfiguracijama. Neke se prodaju u mjerama `1 1/9 bushel`, neke u mjerama `1/2 bushel`, neke po bundevi, neke po funti, a neke u velikim kutijama različitih širina.

Provjerimo ovo:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Nevjerojatno!👏

Čini se da je vrlo teško dosljedno vagati bundeve, pa ih filtrirajmo odabirom samo bundeva koje sadrže niz *bushel* u stupcu `Package` i stavimo to u novi podatkovni okvir `new_pumpkins`.


#### dplyr::filter() i stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): stvara podskup podataka koji sadrži samo **redove** koji zadovoljavaju vaše uvjete, u ovom slučaju, bundeve s nizom *bushel* u stupcu `Package`.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): otkriva prisutnost ili odsutnost uzorka u nizu.

Paket [`stringr`](https://github.com/tidyverse/stringr) pruža jednostavne funkcije za uobičajene operacije s nizovima.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Možete vidjeti da smo suzili na otprilike 415 redaka podataka koji sadrže bundeve po bušelu.🤩  
<br>


#### dplyr::case_when()

**Ali čekajte! Ima još nešto za napraviti**

Jeste li primijetili da se količina u bušelu razlikuje po redovima? Trebate normalizirati cijene tako da prikazujete cijenu po bušelu, a ne po 1 1/9 ili 1/2 bušela. Vrijeme je za malo matematike kako biste to standardizirali.

Koristit ćemo funkciju [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) za *modifikaciju* stupca Price ovisno o određenim uvjetima. `case_when` omogućuje vektorizaciju više `if_else()` izraza.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Sada možemo analizirati cijenu po jedinici na temelju njihove mjere u bušelima. Sve ovo proučavanje bušela bundeva, međutim, pokazuje koliko je `važno` `razumjeti prirodu svojih podataka`!

> ✅ Prema [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308), težina bušela ovisi o vrsti proizvoda, budući da je to mjera volumena. "Bušel rajčica, na primjer, trebao bi težiti 56 funti... Lišće i zelje zauzimaju više prostora s manje težine, pa bušel špinata teži samo 20 funti." Sve je to prilično komplicirano! Nemojmo se zamarati pretvaranjem bušela u funte, već radije odredimo cijenu po bušelu. Sve ovo proučavanje bušela bundeva, međutim, pokazuje koliko je važno razumjeti prirodu svojih podataka!
>
> ✅ Jeste li primijetili da su bundeve koje se prodaju na pola bušela vrlo skupe? Možete li otkriti zašto? Savjet: male bundeve su puno skuplje od velikih, vjerojatno zato što ih ima mnogo više po bušelu, s obzirom na neiskorišteni prostor koji zauzima jedna velika šuplja pita bundeva.


Sada, za čisti užitak avanture 💁‍♀️, premjestimo stupac Mjesec na prvu poziciju, tj. `prije` stupca `Paket`.

`dplyr::relocate()` koristi se za promjenu pozicija stupaca.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Dobar posao!👌 Sada imate čist i uredan skup podataka na kojem možete izgraditi svoj novi regresijski model!


## 4. Vizualizacija podataka s ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Infografika autora Dasani Madipalli</figcaption>


<!--![Infografika autora Dasani Madipalli](../../../../../../2-Regression/2-Data/images/data-visualization.png){width="600"}-->

Postoji jedna *mudra* izreka koja kaže:

> "Jednostavan graf donio je više informacija analitičaru podataka nego bilo koji drugi alat." --- John Tukey

Dio uloge znanstvenika za podatke je pokazati kvalitetu i prirodu podataka s kojima rade. Da bi to postigli, često kreiraju zanimljive vizualizacije, poput grafova, dijagrama i tablica, koje prikazuju različite aspekte podataka. Na taj način mogu vizualno prikazati odnose i praznine koje bi inače bilo teško otkriti.

Vizualizacije također mogu pomoći u određivanju tehnike strojnog učenja koja je najprikladnija za određene podatke. Na primjer, raspršeni graf koji slijedi liniju može ukazivati na to da su podaci dobar kandidat za linearnu regresiju.

R nudi nekoliko sustava za izradu grafova, ali [`ggplot2`](https://ggplot2.tidyverse.org/index.html) je jedan od najelegantnijih i najsvestranijih. `ggplot2` omogućuje sastavljanje grafova **kombiniranjem neovisnih komponenti**.

Započnimo s jednostavnim raspršenim grafom za stupce Price i Month.

U ovom slučaju, počet ćemo s [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), dodati skup podataka i estetsko mapiranje (pomoću [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)), a zatim dodati slojeve (poput [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) za raspršene grafove.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Je li ovo koristan grafikon 🤷? Iznenađuje li vas nešto u vezi s njim?

Nije osobito koristan jer samo prikazuje vaše podatke kao raspršene točke u određenom mjesecu.  
<br>


### **Kako učiniti podatke korisnima?**

Kako bismo prikazali korisne podatke na grafikonima, obično je potrebno nekako grupirati podatke. Na primjer, u našem slučaju, pronalaženje prosječne cijene bundeva za svaki mjesec pružilo bi više uvida u osnovne obrasce u našim podacima. To nas dovodi do još jednog brzog pregleda **dplyr**:

#### `dplyr::group_by() %>% summarize()`

Grupirana agregacija u R-u može se lako izračunati pomoću

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` mijenja jedinicu analize s cijelog skupa podataka na pojedinačne grupe, poput grupiranja po mjesecima.

-   `dplyr::summarize()` stvara novi podatkovni okvir s jednim stupcem za svaku grupirajuću varijablu i jednim stupcem za svaku od statistika sažetka koje ste odredili.

Na primjer, možemo koristiti `dplyr::group_by() %>% summarize()` kako bismo grupirali bundeve u grupe na temelju stupca **Month** i zatim pronašli **prosječnu cijenu** za svaki mjesec.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Sažeto!✨

Kategorijske značajke, poput mjeseci, bolje su prikazane pomoću stupčastog grafikona 📊. Slojevi odgovorni za stupčaste grafikone su `geom_bar()` i `geom_col()`. Pogledajte `?geom_bar` za više informacija.

Idemo napraviti jedan!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩Ovo je korisnija vizualizacija podataka! Čini se da pokazuje kako su najviše cijene bundeva u rujnu i listopadu. Odgovara li to vašim očekivanjima? Zašto da ili zašto ne?

Čestitamo na završetku druge lekcije 👏! Pripremili ste svoje podatke za izgradnju modela, a zatim otkrili više uvida koristeći vizualizacije!



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane čovjeka. Ne preuzimamo odgovornost za bilo kakva nesporazuma ili pogrešna tumačenja koja proizlaze iz korištenja ovog prijevoda.
